# Analysis of quasi-elastic neutron scattering (QENS) measurement of a protein in liquid state

This file contains the code of the nPDyn documentation available here: https://npdyn.readthedocs.io/en/latest/.


In [1]:
import numpy as np

from nPDyn import Dataset

In [2]:
qens = Dataset(
    QENSFiles=['sample_data/lys_part_01_QENS_before_280K.nxs'],
    resFiles=['sample_data/vana_QENS_280K.nxs'],
    ECFile='sample_data/empty_cell_QENS_280K.nxs',
    D2OFile='sample_data/D2O_QENS_280K.nxs')

efws = Dataset(
    FWSFiles=['sample_data/D_syn_fibers_elastic_10to300K.inx'])

## Process the data

In [3]:
qens.binAll(5)
qens.subtract_EC()
qens.setQRange(0.4, 1.8)
q = qens.dataList[0].data.qVals[:, np.newaxis]

efws.binAll(20, 'observable')
efws.normalize_usingLowTemp(nbrBins=6)
efws.setQRange(0.2, 1)

## For QENS, fit the resolution function and the D2O background

In [4]:
from nPDyn.models.builtins import modelD2OBackground, modelPVoigt

qens.fitRes(model=modelPVoigt(q, 'resolution'))
qens.D2OData.fit(model=modelD2OBackground(q, temperature=280))

Fitting dataset: sample_data/vana_QENS_280K.nxs
	Fit of observable 1 of 1 (time=0.0)
Done.

Fitting dataset: sample_data/D2O_QENS_280K.nxs
	Fit of observable 1 of 1 (time=0.0)
Done.



## Create the parameters

In [5]:
from nPDyn.models import Parameters

pQENS = Parameters(
    beta={'value': np.zeros_like(q) + 1, 'bounds': (0., np.inf)},
    alpha={'value': 0.5, 'bounds': (0., 1)},
    Ds={'value': 5, 'bounds': (0., 100)},
    Di={'value': 20, 'bounds': (0., 100)},
    tau={'value': 1, 'bounds': (0., np.inf)},
    bD2O={'value': np.zeros_like(q) + 0.1, 'bounds': (0., np.inf)})

pEFWS = Parameters(scale=1, msd=0.5)
pEFWS.set('scale', bounds=(0., np.inf), fixed=False)
pEFWS.set('msd', bounds=(0., np.inf))

## Create the models

In [6]:
from nPDyn.models import Model

modelQENS = Model(pQENS, 'QENS')
modelEFWS = Model(pEFWS, 'EFWS')

## Add the components to the models

In [7]:
from nPDyn.models import Component
from nPDyn.models.presets import lorentzian

modelQENS.addComponent(Component(
    'center-of-mass', 
    lorentzian,
    scale='beta * alpha',  # will find the parameters values in pQENS
    width='Ds * q**2',  # we will give q on the call to the fit method
    center=0))  # we force the center to be at 0 
                # (as it is given by the convolution with resolution)
# we can add, subtract, multiply or divide a model using a Component o
# another Model 
internal = Component(
    'internal', 
    lorentzian,
    scale='beta * (1 - alpha)', 
    width='Di * q**2 / (1 + Di * q**2 * tau) + Ds * q**2', 
    center=0)  # we force the center to be at 0 
               # (as it is given by the convolution with resolution)
modelQENS += internal
# for the D2O signal, we use a lambda function to include the scaling
modelQENS.addComponent(Component(
    '$D_2O$',
    lambda x, scale: scale * qens.D2OData.fit_best(x=x)[0], 
    scale='bD2O',
    skip_convolve=True))

modelEFWS.addComponent(Component(
    'EISF',
    lambda x, scale, msd: scale * np.exp(-x**2 * msd / 6)))

## Fit the data

In [8]:
qens.fitData(
    model=modelQENS, convolveRes=True, q=q,
    fit_method='basinhopping', fit_kws={'niter': 25, 'disp': True})

efws.fitData(model=modelEFWS, x=efws.dataList[0].data.qVals[:, np.newaxis])

Fitting dataset: sample_data/lys_part_01_QENS_before_280K.nxs
basinhopping step 0: f 1083.75e=0.0)
basinhopping step 1: f 1083.75 trial_f 1083.75 accepted 1  lowest_f 1083.75
found new global minimum on step 1 with function value 1083.75
basinhopping step 2: f 1083.75 trial_f 1083.75 accepted 1  lowest_f 1083.75
basinhopping step 3: f 1083.75 trial_f 1083.75 accepted 1  lowest_f 1083.75
basinhopping step 4: f 1083.75 trial_f 1083.75 accepted 1  lowest_f 1083.75
basinhopping step 5: f 1083.75 trial_f 1083.75 accepted 1  lowest_f 1083.75
found new global minimum on step 5 with function value 1083.75
basinhopping step 6: f 1083.75 trial_f 1083.75 accepted 1  lowest_f 1083.75
found new global minimum on step 6 with function value 1083.75
basinhopping step 7: f 1083.75 trial_f 1083.75 accepted 1  lowest_f 1083.75
basinhopping step 8: f 1083.75 trial_f 1083.75 accepted 1  lowest_f 1083.75
basinhopping step 9: f 1083.77 trial_f 1083.77 accepted 1  lowest_f 1083.75
basinhopping step 10: f 1083

## Plot the data

In [9]:
qens.plotQENS()

In [10]:
efws.plotFWS()